### Dreambooth Fine-tuning
DreamBooth is a deep learning generation model used to fine-tune existing text-to-image models, developed by researchers from Google Research and Boston University in 2022. Originally developed using Google's own Imagen text-to-image model, DreamBooth implementations can be applied to other text-to-image models, where it can allow the model to generate more fine-tuned and personalised outputs after training on three to five images of a subject.

We should use dreambooth fine tuning our stable diffusion model.

#### Notebook step
1. Import boto3, sagemaker python SDK
2. Build dreambooth fine-tuning image
3. Fine-tuning 
   * config hyperparameter
   * create training job
4. Testing 

#### 1. Import boto3, sagemaker python SDK

In [1]:
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

images_s3uri = 's3://{0}/dreambooth/images/'.format(bucket)
models_s3uri = 's3://{0}/stable-diffusion/models/'.format(bucket)
dreambooth_s3uri = 's3://{0}/stable-diffusion/dreambooth/'.format(bucket)

print(bucket)

sagemaker-us-east-1-462188225149


In [5]:
images_s3uri

's3://sagemaker-us-east-2-462188225149/dreambooth/images/'

In [21]:
models_s3uri = 's3://sagemaker-ap-northeast-1-462188225149/RVSD2/'
models_s3uri

's3://sagemaker-ap-northeast-1-462188225149/RVSD2/'

In [13]:
ls -al

total 124
drwxrwxr-x 3 ec2-user ec2-user  4096 Mar 27 06:53 ./
drwxrwxr-x 4 ec2-user ec2-user  4096 Mar 27 06:28 ../
-rwxrwxr-x 1 ec2-user ec2-user  1303 Mar 27 06:28 build_push.sh*
-rw-rw-r-- 1 ec2-user ec2-user  1330 Mar 27 06:28 Dockerfile
-rw-rw-r-- 1 ec2-user ec2-user   376 Mar 27 06:28 Dockerfile.public-ecr
drwxrwxr-x 2 ec2-user ec2-user  4096 Mar 27 06:28 .ipynb_checkpoints/
-rw-rw-r-- 1 ec2-user ec2-user   204 Mar 27 06:28 README.md
-rw-rw-r-- 1 ec2-user ec2-user 12849 Mar 27 06:53 stablediffusion_dreambooth_finetuning.en.ipynb
-rw-rw-r-- 1 ec2-user ec2-user  7505 Mar 27 06:28 stablediffusion_dreambooth_finetuning_notebook.zh.ipynb
-rw-rw-r-- 1 ec2-user ec2-user  8757 Mar 27 06:28 stablediffusion_dreambooth_finetuning.zh.ipynb
-rw-rw-r-- 1 ec2-user ec2-user 54493 Mar 27 06:28 train_dreambooth.py
-rwxrwxr-x 1 ec2-user ec2-user  1432 Mar 27 06:28 train.sh*


In [37]:
!chmod +x build_push.sh

In [2]:
!curl https://dfjcgkift2mhn.cloudfront.net/aigc/images.tgz | tar -xz 
!ls images/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1297k  100 1297k    0     0  18.1M      0 --:--:-- --:--:-- --:--:-- 18.3M
image-1.jpg  image-2.jpg  image-3.jpg  image-6.jpg  image-7.jpg  image-8.jpg


In [26]:
rm -rf ./sd_code

#### 2. Build dreambooth fine-tuning image 
  It will take 60~90 minutes if use small notebook instance(ml.t3.xlarge)

In [38]:
!./build_push.sh

Using default tag: latest
latest: Pulling from o7x6j3x6/sd-dreambooth-finetuning-v2
Digest: sha256:0140c9f9650458d39f63c3d4ada64eee96984b955117bea59db9749f031b3403
Status: Image is up to date for public.ecr.aws/o7x6j3x6/sd-dreambooth-finetuning-v2:latest
public.ecr.aws/o7x6j3x6/sd-dreambooth-finetuning-v2:latest
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Cloning into 'sd_dreambooth_extension'...
remote: Enumerating objects: 1951, done.
remote: Counting objects: 100% (189/189), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 1951 (delta 119), reused 167 (delta 104), pack-reused 1762
Receiving objects: 100% (1951/1951), 10.25 MiB | 26.65 MiB/s, done.
Resolving deltas: 100% (1233/1233), done.
Sending 

#### 3. Fine-tuning 

   * image_uri: docker image ecr URI
   * instance_type: training job instance , prefere ml.g4dn.xlarge, ml.g5.xlarge
   * class_prompt: class prompt
   * instance_prompt: your image key prompt
   * model_name: pretrained_model 
   

In [39]:
import json
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {k: json.dumps(v) for (k, v) in hyperparameters.items()}




image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sd-dreambooth-finetuning-v2'
instance_type = 'ml.g4dn.2xlarge'

instance_prompt="photo\ of\ zwx\  man"
class_prompt="photo\ of\ a\ man"
s3_model_output_location='s3://{}/{}/{}'.format(bucket, 'dreambooth', 'trained_models')
model_name="runwayml/stable-diffusion-v1-5"
# model_name="SG161222/Realistic_Vision_V1.4"
instance_dir="/opt/ml/input/data/images/"
class_dir="/opt/ml/input/data/class_images/"



environment = {
    'PYTORCH_CUDA_ALLOC_CONF':'max_split_size_mb:32',
    'LD_LIBRARY_PATH':"${LD_LIBRARY_PATH}:/opt/conda/lib/"
}

hyperparameters = {
                    'model_name':'aws-trained-dreambooth-model',
                    'mixed_precision':'fp16',
                    'pretrained_model_name_or_path': model_name, 
                    'instance_data_dir':instance_dir,
                    'class_data_dir':class_dir,
                    'with_prior_preservation':False,
                    'models_path': '/opt/ml/model/',
                    'instance_prompt': instance_prompt, 
                    'class_prompt':class_prompt,
                    'resolution':512,
                    'train_batch_size':1,
                    'sample_batch_size': 1,
                    'gradient_accumulation_steps':1,
                    'learning_rate':2e-06,
                    'lr_scheduler':'constant',
                    'lr_warmup_steps':0,
                    'num_class_images':50,
                    'max_train_steps':10,
                    'save_steps':300,
                    'attention':'xformers',
                    'prior_loss_weight': 0.5,
                    'use_ema':True,
                    'train_text_encoder':False,
                    'not_cache_latents':True,
                    'gradient_checkpointing':True,
                    'save_use_epochs': False,
                    'use_8bit_adam': False,
                    's3_output': s3_model_output_location
}

hyperparameters = json_encode_hyperparameters(hyperparameters)



model_name aws-trained-dreambooth-model
mixed_precision fp16
pretrained_model_name_or_path runwayml/stable-diffusion-v1-5
instance_data_dir /opt/ml/input/data/images/
class_data_dir /opt/ml/input/data/class_images/
with_prior_preservation False
models_path /opt/ml/model/
instance_prompt photo\ of\ zwx\  man
class_prompt photo\ of\ a\ man
resolution 512
train_batch_size 1
sample_batch_size 1
gradient_accumulation_steps 1
learning_rate 2e-06
lr_scheduler constant
lr_warmup_steps 0
num_class_images 50
max_train_steps 10
save_steps 300
attention xformers
prior_loss_weight 0.5
use_ema True
train_text_encoder False
not_cache_latents True
gradient_checkpointing True
save_use_epochs False
use_8bit_adam False
s3_output s3://sagemaker-us-east-1-462188225149/dreambooth/trained_models


   * Create training job 

In [40]:
from sagemaker.estimator import Estimator
inputs = {
    'images': f"s3://sagemaker-{region_name}-{account_id}/dreambooth/images/"
}


estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters,
    environment = environment,
)
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: sd-dreambooth-finetuning-v2-2023-04-01-12-38-45-038


2023-04-01 12:38:45 Starting - Starting the training job...
2023-04-01 12:39:02 Starting - Preparing the instances for training......
2023-04-01 12:40:12 Downloading - Downloading input data
2023-04-01 12:40:12 Training - Downloading the training image.................................
2023-04-01 12:45:23 Training - Training image download completed. Training in progress..2023-04-01 12:45:50,508 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-01 12:45:50,539 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-01 12:45:50,569 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-04-01 12:45:50,580 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "images": "/opt/ml/input/data/images"
    },
    "current_host": "algo-1",
    "current_instance_group": "homog

In [19]:
dreambooth_model_data = estimator.model_data
print("Model artifact saved at:\n", dreambooth_model_data)

Model artifact saved at:
 s3://sagemaker-us-east-1-462188225149/dreambooth/trained_models/sd-dreambooth-finetuning-v2-2023-04-01-08-37-05-153/output/model.tar.gz


#### 4. Testing 
  you can use inference notebook load your new model

In [ ]:
import boto3
import sagemaker
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()



In [33]:
print(f'execution role: {role}')
print(f'default bucket: {bucket}')

execution role: arn:aws:iam::462188225149:role/SagemakerSwain
default bucket: sagemaker-us-east-1-462188225149


In [ ]:

framework_version = '1.10'
py_version = 'py38'

model_environment = {
    'SAGEMAKER_MODEL_SERVER_TIMEOUT':'600', 
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 
    'model_name':'Linaqruf/anything-v3.0', #huggingface model name 
    's3_bucket':  bucket
}

In [ ]:
!touch dummy
!tar czvf model.tar.gz dummy sagemaker-logo-small.png
assets_dir = 's3://{0}/{1}/assets/'.format(bucket, 'stablediffusion')
model_data = 's3://{0}/{1}/assets/model.tar.gz'.format(bucket, 'stablediffusion')
!aws s3 cp model.tar.gz $assets_dir
!rm -f dummy model.tar.gz

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

model = PyTorchModel(
    name = None,
    model_data = model_data,
    entry_point = 'inference.py',
    source_dir = "./code/",
    role = role,
    framework_version = framework_version, 
    py_version = py_version,
    env = model_environment
)

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig
import uuid

endpoint_name = f'AIGC-Quick-Kit-{str(uuid.uuid4())}'
instance_type = 'ml.g4dn.xlarge'
instance_count = 1
async_config = AsyncInferenceConfig(output_path='s3://{0}/{1}/asyncinvoke/out/'.format(bucket, 'stablediffusion'))

print(f'endpoint_name: {endpoint_name}')

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


async_predictor = model.deploy(
    endpoint_name = endpoint_name,
    instance_type = instance_type, 
    initial_instance_count = instance_count,
    async_inference_config = async_config,
    serializer = JSONSerializer(),
    deserializer = JSONDeserializer()
)